# 0. 라이브러리 불러오기

In [1]:
import mido
from mido import MidiFile, MidiTrack
import numpy as np
import pandas as pd
import editdistance
import os

# 1. 동일한 시간에 동시에 누르는 건반에 대해서 데이터 추출하기
- 'Sec' (초): 각 미디 이벤트가 발생한 시간
- 'Ticks' (틱): MIDI 파일에서 사용되는 시간 단위인 틱
- 'msg_type' (이벤트 타입): 각 미디 이벤트의 종류(주로 'note_on', 'note_off', 'set_tempo' 등의 값)
- 'note_on' 이벤트는 악기에서 음표를 연주할 때 발생하는 이벤트로, 특정 음표가 시작되는 시점을 나타냄
- 'note_off' 이벤트는 해당 음표의 연주가 끝나는 시점을 나타냄
- 'set_tempo': 'set_tempo' 이벤트는 MIDI 파일 내에서 템포(속도)를 설정하는 이벤트, 템포는 음악의 속도를 나타내며, 'set_tempo' 이벤트를 통해 BPM(분당 박자) 값이 설정됨
- 'Channel' (채널): MIDI에서 사용되는 채널 번호(미디 이벤트가 어떤 악기 채널에 속하는지를 표시)
- 'Notes' (음표): 노트 이벤트에서 발생한 음표(음 높이) 정보
- 'Velocities' (속도): 노트 이벤트에서 발생한 음표의 속도(음의 세기) 정보
- 'Tempo' (템포): 템포 이벤트에서 설정된 BPM (분당 박자) 값
- 'diff_velocity_mean' (속도 변화): 각 노트 이벤트에서의 속도의 차이(현재 노트의 평균 속도 - 이전 노트의 평균 속도)
    - 음표의 세기, 빠르기 변화 등에 활용

In [562]:
def midi_to_dataframe(file_path):
    mid = mido.MidiFile(file_path)

    columns = ['Sec', 'Ticks', 'msg_type', 'Channel', 'Notes', 'Velocities', 'Tempo', 'mean_velocity', 'diff_velocity']
    midi_df = pd.DataFrame(columns=columns)

    current_notes = []
    current_velocities = []
    current_tempo = None
    current_ticks = None
    current_time = 0
    current_msg_type = []
    prev_notes = []
    prev_velocities = []

    for msg in mid:
        if msg.type in ['note_on', 'note_off']:
            if msg.time > 0:
                # 이전 노트 및 velocity의 평균 계산
                prev_avg_velocity = sum(prev_velocities) / len(prev_velocities) if prev_velocities else 0

                # 변화 속도 및 평균 속도 계산
                if len(current_velocities) > 0:  # 노트가 있는 경우에만 계산
                    diff_velocity = sum(current_velocities) / len(current_velocities) - prev_avg_velocity
                    mean_velocity = sum(current_velocities) / len(current_velocities)
                else:
                    diff_velocity = 0  # 없으면 0으로 처리
                    mean_velocity = 0

                if current_msg_type:
                    midi_df = pd.concat([midi_df, pd.DataFrame([[current_time, current_ticks, current_msg_type, msg.channel, current_notes,
                                                                 current_velocities, current_tempo, diff_velocity, mean_velocity]], columns=columns)], ignore_index=True)

                prev_notes = current_notes.copy()
                prev_velocities = current_velocities.copy()

                current_notes = []
                current_velocities = []
                current_msg_type = []

            current_notes.append(msg.note)
            current_velocities.append(msg.velocity)
            current_msg_type.append(msg.type)

        elif msg.type == 'set_tempo':
            current_tempo = mido.tempo2bpm(msg.tempo)  # 템포 정보 업데이트

        current_ticks = msg.time

        current_time += msg.time

    # 마지막 노트들을 처리
    if current_notes and current_msg_type:
        prev_avg_velocity = sum(prev_velocities) / len(prev_velocities) if prev_velocities else 0
        if len(current_velocities) > 0:  # 노트가 있는 경우에만 계산
            diff_velocity = sum(current_velocities) / len(current_velocities) - prev_avg_velocity
            mean_velocity = sum(current_velocities) / len(current_velocities)
        else:
            diff_velocity = 0
            mean_velocity = 0

        midi_df = pd.concat([midi_df, pd.DataFrame([[current_time, current_ticks, current_msg_type, '', current_notes,
                                                     current_velocities, current_tempo, diff_velocity, mean_velocity]], columns=columns)], ignore_index=True)

    return midi_df

In [563]:
def print_midi_info(file_path):
    mid = mido.MidiFile(file_path)

    print("파일이름 :", mid.filename)
    print("총 재생시간 :", mid.length)

    if mid.tracks:
        print('-' * 100)
        print("트랙 이름 :", mid.tracks[0].name)

        # mid.tracks의 길이를 통해 총 트랙의 수 확인
        total_tracks = len(mid.tracks)
        print("총 트랙의 수:", total_tracks)
        for i in range(total_tracks):
            if mid.tracks[i].name:
                print(mid.tracks[i].name)  # 바이트값 출력
            else:
                print("트랙 이름이 없음")

        print('-' * 100)

        for i in range(total_tracks):
            note_count = sum(1 for msg in mid.tracks[i] if msg.type == 'note_on' or msg.type == 'note_off')
            print(f"트랙 {i + 1}의 총 노트 수: {note_count}")
        print('-' * 100)

        for msg in mid:
            print(msg)

    else:
        print("트랙이 없습니다.")

In [564]:
midi_file_path = 'midi_data/Fugue1.mid'

# 미디 파일의 트랙 정보 출력
print_midi_info(midi_file_path)

result_df = midi_to_dataframe(midi_file_path)
result_df.to_csv('hello1.csv', index=False)

print(result_df)

파일이름 : midi_data/Fugue1.mid
총 재생시간 : 103.13932137500001
----------------------------------------------------------------------------------------------------
트랙 이름 : 
총 트랙의 수: 5
트랙 이름이 없음
트랙 이름이 없음
트랙 이름이 없음
트랙 이름이 없음
트랙 이름이 없음
----------------------------------------------------------------------------------------------------
트랙 1의 총 노트 수: 0
트랙 2의 총 노트 수: 464
트랙 3의 총 노트 수: 396
트랙 4의 총 노트 수: 350
트랙 5의 총 노트 수: 288
----------------------------------------------------------------------------------------------------
MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0)
MetaMessage('set_tempo', tempo=937500, time=0)
MetaMessage('text', text='1', time=0)
MetaMessage('text', text='2', time=0)
MetaMessage('text', text='3', time=0)
MetaMessage('text', text='4', time=0)
note_on channel=1 note=60 velocity=64 time=0.46875
note_off channel=1 note=60 velocity=62 time=0.46875
note_on channel=1 note=62 velocity=64 time=0
note_off channel=1

            Sec     Ticks                      msg_type Channel         Notes  \
0      0.468750  0.468750                     [note_on]       1          [60]   
1      0.937500  0.000000           [note_off, note_on]       1      [60, 62]   
2      1.406250  0.000000           [note_off, note_on]       1      [62, 64]   
3      1.875000  0.000000           [note_off, note_on]       1      [64, 65]   
4      2.578125  0.000000           [note_off, note_on]       1      [65, 67]   
..          ...       ...                           ...     ...           ...   
765   99.981427  0.000000  [note_off, note_on, note_on]       3  [74, 79, 76]   
766  102.961690  2.980262                    [note_off]       1          [48]   
767  103.001164  0.039474                    [note_off]       0          [60]   
768  103.119585  0.118421                    [note_off]       1          [84]   
769  103.139321  0.000000          [note_off, note_off]              [76, 79]   

       Velocities      Temp

In [565]:
midi_file_path = 'midi_data/Fugue3.mid'

# 미디 파일의 트랙 정보 출력
print_midi_info(midi_file_path)

result_df = midi_to_dataframe(midi_file_path)
result_df.to_csv('hello2.csv', index=False)

print(result_df)

파일이름 : midi_data/Fugue3.mid
총 재생시간 : 162.20662299999861
----------------------------------------------------------------------------------------------------
트랙 이름 : 
총 트랙의 수: 4
트랙 이름이 없음
트랙 이름이 없음
트랙 이름이 없음
트랙 이름이 없음
----------------------------------------------------------------------------------------------------
트랙 1의 총 노트 수: 0
트랙 2의 총 노트 수: 1218
트랙 3의 총 노트 수: 670
트랙 4의 총 노트 수: 1004
----------------------------------------------------------------------------------------------------
MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0)
MetaMessage('set_tempo', tempo=731707, time=0)
MetaMessage('text', text='RH H', time=0)
MetaMessage('text', text='RH L', time=0)
MetaMessage('text', text='LH L', time=0)
note_on channel=0 note=68 velocity=64 time=1.0975605
note_off channel=0 note=68 velocity=80 time=0.3658535
note_on channel=0 note=70 velocity=64 time=0
note_off channel=0 note=70 velocity=74 time=0.18292675
note_on channe

            Sec     Ticks                                           msg_type  \
0      1.097560  1.097560                                          [note_on]   
1      1.463414  0.000000                                [note_off, note_on]   
2      1.646341  0.000000                                [note_off, note_on]   
3      1.829268  0.000000                                [note_off, note_on]   
4      2.012194  0.000000                                [note_off, note_on]   
..          ...       ...                                                ...   
892  159.584519  0.000000                     [note_off, note_off, note_off]   
893  159.723864  0.098361               [note_on, note_on, note_on, note_on]   
894  160.094554  0.000000  [note_off, note_off, note_off, note_off, note_...   
895  160.137657  0.000000      [note_on, note_on, note_on, note_on, note_on]   
896  162.206623  0.000000  [note_off, note_off, note_off, note_off, note_...   

    Channel                 Notes      

In [566]:
midi_file_path = 'midi_data/Prelude1.mid'

# 미디 파일의 트랙 정보 출력
print_midi_info(midi_file_path)

result_df = midi_to_dataframe(midi_file_path)
result_df.to_csv('hello3.csv', index=False)

print(result_df)

파일이름 : midi_data/Prelude1.mid
총 재생시간 : 117.95786391249983
----------------------------------------------------------------------------------------------------
트랙 이름 : 
총 트랙의 수: 3
트랙 이름이 없음
트랙 이름이 없음
트랙 이름이 없음
----------------------------------------------------------------------------------------------------
트랙 1의 총 노트 수: 0
트랙 2의 총 노트 수: 834
트랙 3의 총 노트 수: 268
----------------------------------------------------------------------------------------------------
MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0)
MetaMessage('set_tempo', tempo=833333, time=0)
MetaMessage('text', text='RH B', time=0)
MetaMessage('text', text='LH B', time=0)
note_on channel=0 note=60 velocity=64 time=0
note_on channel=0 note=64 velocity=64 time=0.20833325
note_on channel=0 note=67 velocity=64 time=0.20833325
note_off channel=0 note=67 velocity=44 time=0.20833325
note_on channel=0 note=72 velocity=64 time=0
note_off channel=0 note=72 velocity=7

            Sec     Ticks                        msg_type Channel  \
0             0         0                       [note_on]       0   
1      0.208333  0.208333                       [note_on]       0   
2      0.416666  0.208333                       [note_on]       0   
3         0.625         0             [note_off, note_on]       0   
4      0.833333         0             [note_off, note_on]       0   
..          ...       ...                             ...     ...   
725  114.419671  0.013889                      [note_off]       0   
726  114.579393  0.159722                      [note_off]       0   
727  114.624532         0              [note_on, note_on]       0   
728  117.322447         0  [note_off, note_off, note_off]       0   
729  117.957864         0            [note_off, note_off]           

            Notes    Velocities      Tempo  mean_velocity  diff_velocity  
0            [60]          [64]  72.000029      64.000000      64.000000  
1            [64]    

In [567]:
midi_file_path = 'midi_data/sample_midi.mid'

# 미디 파일의 트랙 정보 출력
print_midi_info(midi_file_path)

result_df = midi_to_dataframe(midi_file_path)
result_df.to_csv('hello4.csv', index=False)

print(result_df)

파일이름 : midi_data/sample_midi.mid
총 재생시간 : 59.000000000000014
----------------------------------------------------------------------------------------------------
트랙 이름 : Chill The World Chords
총 트랙의 수: 1
Chill The World Chords
----------------------------------------------------------------------------------------------------
트랙 1의 총 노트 수: 152
----------------------------------------------------------------------------------------------------
MetaMessage('channel_prefix', channel=0, time=2.0)
MetaMessage('track_name', name='Chill The World Chords', time=0)
MetaMessage('instrument_name', name='Chill The World Chords', time=0)
MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0)
MetaMessage('key_signature', key='C', time=0)
MetaMessage('smpte_offset', frame_rate=25, hours=1, minutes=0, seconds=0, frames=0, sub_frames=0, time=0)
MetaMessage('set_tempo', tempo=750000, time=0)
note_on channel=0 note=41 velocity=70 time=0
note_

In [568]:
midi_file_path = 'midi_data/hangyul_input.midi'

# 미디 파일의 트랙 정보 출력
print_midi_info(midi_file_path)

result_df = midi_to_dataframe(midi_file_path)
result_df.to_csv('input.csv', index=False)

print(result_df)

파일이름 : midi_data/hangyul_input.midi
총 재생시간 : 174.7791666666652
----------------------------------------------------------------------------------------------------
트랙 이름 : test_rivers_flow
총 트랙의 수: 4
test_rivers_flow
MIDI
Piano
Yamaha MOX
----------------------------------------------------------------------------------------------------
트랙 1의 총 노트 수: 0
트랙 2의 총 노트 수: 0
트랙 3의 총 노트 수: 2564
트랙 4의 총 노트 수: 0
----------------------------------------------------------------------------------------------------
MetaMessage('track_name', name='test_rivers_flow', time=0)
MetaMessage('set_tempo', tempo=1000000, time=0)
MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0)
MetaMessage('track_name', name='MIDI', time=0)
MetaMessage('key_signature', key='A', time=0)
MetaMessage('track_name', name='Piano', time=0)
MetaMessage('track_name', name='Yamaha MOX', time=0)
note_on channel=0 note=80 velocity=36 time=0.5270833333333333
note_on cha

control_change channel=0 control=64 value=96 time=0.008333333333333333
control_change channel=0 control=64 value=96 time=0
control_change channel=0 control=64 value=88 time=0.010416666666666666
control_change channel=0 control=64 value=88 time=0
control_change channel=0 control=64 value=80 time=0.010416666666666666
control_change channel=0 control=64 value=80 time=0
control_change channel=0 control=64 value=76 time=0.0125
control_change channel=0 control=64 value=76 time=0
control_change channel=0 control=64 value=74 time=0.00625
control_change channel=0 control=64 value=74 time=0
control_change channel=0 control=64 value=76 time=0.010416666666666666
control_change channel=0 control=64 value=76 time=0.0020833333333333333
control_change channel=0 control=64 value=80 time=0.010416666666666666
control_change channel=0 control=64 value=80 time=0
control_change channel=0 control=64 value=90 time=0.008333333333333333
control_change channel=0 control=64 value=90 time=0
control_change channel=

             Sec     Ticks            msg_type Channel     Notes Velocities  \
0       0.527083  0.527083           [note_on]       0      [80]       [36]   
1       0.529167  0.002083           [note_on]       0      [80]       [36]   
2       0.537500  0.008333           [note_on]       0      [61]       [17]   
3       0.539583  0.002083           [note_on]       0      [61]       [17]   
4       0.843750  0.000000  [note_on, note_on]       0  [61, 61]     [0, 0]   
...          ...       ...                 ...     ...       ...        ...   
1640  174.058333  0.000000  [note_on, note_on]       0  [57, 57]     [0, 0]   
1641  174.075000  0.000000  [note_on, note_on]       0  [81, 81]     [0, 0]   
1642  174.218750  0.000000  [note_on, note_on]       0  [62, 62]     [0, 0]   
1643  174.277083  0.000000  [note_on, note_on]       0  [69, 69]   [30, 30]   
1644  174.779167  0.000000  [note_on, note_on]          [69, 69]     [0, 0]   

      Tempo  mean_velocity  diff_velocity  
0      

In [569]:
midi_file_path = 'midi_data/hangyul_target.mid'

# 미디 파일의 트랙 정보 출력
print_midi_info(midi_file_path)

result_df = midi_to_dataframe(midi_file_path)
result_df.to_csv('target.csv', index=False)

print(result_df)

파일이름 : midi_data/hangyul_target.mid
총 재생시간 : 164.77394462499913
----------------------------------------------------------------------------------------------------
트랙 이름 : 
총 트랙의 수: 2
트랙 이름이 없음
Piano
----------------------------------------------------------------------------------------------------
트랙 1의 총 노트 수: 0
트랙 2의 총 노트 수: 1600
----------------------------------------------------------------------------------------------------
MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0)
MetaMessage('key_signature', key='A', time=0)
MetaMessage('set_tempo', tempo=923075, time=0)
control_change channel=0 control=0 value=0 time=0
control_change channel=0 control=32 value=0 time=0
program_change channel=0 program=0 time=0
MetaMessage('track_name', name='Piano', time=0)
note_on channel=0 note=81 velocity=70 time=0
note_on channel=0 note=54 velocity=72 time=0
note_on channel=0 note=81 velocity=0 time=0.4615375
note_on channel=0 

            Sec     Ticks                                           msg_type  \
0             0         0                                 [note_on, note_on]   
1      0.461537         0               [note_on, note_on, note_on, note_on]   
2      0.923075         0               [note_on, note_on, note_on, note_on]   
3      1.384613         0                                 [note_on, note_on]   
4       1.84615         0               [note_on, note_on, note_on, note_on]   
..          ...       ...                                                ...   
516  158.076259         0                                 [note_on, note_on]   
517  158.537796         0               [note_on, note_on, note_on, note_on]   
518  159.460869         0               [note_on, note_on, note_on, note_on]   
519  160.383942         0  [note_on, note_on, note_on, note_on, note_on, ...   
520  164.773945  0.005409  [note_on, note_on, note_on, note_on, note_on, ...   

    Channel                            

In [579]:
midi_file_path = 'midi_data/hangyul_input2_1.midi'

# 미디 파일의 트랙 정보 출력
print_midi_info(midi_file_path)

result_df = midi_to_dataframe(midi_file_path)
result_df.to_csv('input2_1.csv', index=False)

print(result_df)

파일이름 : midi_data/hangyul_input2_1.midi
총 재생시간 : 12.470833333333331
----------------------------------------------------------------------------------------------------
트랙 이름 : test_rivers_flow
총 트랙의 수: 3
test_rivers_flow
Piano
Yamaha MOX
----------------------------------------------------------------------------------------------------
트랙 1의 총 노트 수: 0
트랙 2의 총 노트 수: 108
트랙 3의 총 노트 수: 0
----------------------------------------------------------------------------------------------------
MetaMessage('track_name', name='test_rivers_flow', time=0)
MetaMessage('set_tempo', tempo=1000000, time=0)
MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0)
MetaMessage('track_name', name='Piano', time=0)
MetaMessage('track_name', name='Yamaha MOX', time=0)
note_on channel=0 note=72 velocity=53 time=2.8125
note_on channel=0 note=72 velocity=53 time=0
note_on channel=0 note=76 velocity=75 time=0.32708333333333334
note_on channel=0 note=76 

In [580]:
midi_file_path = 'midi_data/hangyul_input2_2.midi'

# 미디 파일의 트랙 정보 출력
print_midi_info(midi_file_path)

result_df = midi_to_dataframe(midi_file_path)
result_df.to_csv('input2_2.csv', index=False)

print(result_df)

파일이름 : midi_data/hangyul_input2_2.midi
총 재생시간 : 12.458333333333329
----------------------------------------------------------------------------------------------------
트랙 이름 : test_rivers_flow
총 트랙의 수: 3
test_rivers_flow
Piano
Yamaha MOX
----------------------------------------------------------------------------------------------------
트랙 1의 총 노트 수: 0
트랙 2의 총 노트 수: 96
트랙 3의 총 노트 수: 0
----------------------------------------------------------------------------------------------------
MetaMessage('track_name', name='test_rivers_flow', time=0)
MetaMessage('set_tempo', tempo=1000000, time=0)
MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0)
MetaMessage('track_name', name='Piano', time=0)
MetaMessage('track_name', name='Yamaha MOX', time=0)
note_on channel=0 note=72 velocity=59 time=2.7916666666666665
note_on channel=0 note=72 velocity=59 time=0
note_on channel=0 note=76 velocity=68 time=0.32708333333333334
note_on channel

In [581]:
midi_file_path = 'midi_data/hangyul_input2_3.midi'

# 미디 파일의 트랙 정보 출력
print_midi_info(midi_file_path)

result_df = midi_to_dataframe(midi_file_path)
result_df.to_csv('input2_3.csv', index=False)

print(result_df)

파일이름 : midi_data/hangyul_input2_3.midi
총 재생시간 : 12.583333333333334
----------------------------------------------------------------------------------------------------
트랙 이름 : test_rivers_flow
총 트랙의 수: 3
test_rivers_flow
Piano
Yamaha MOX
----------------------------------------------------------------------------------------------------
트랙 1의 총 노트 수: 0
트랙 2의 총 노트 수: 92
트랙 3의 총 노트 수: 0
----------------------------------------------------------------------------------------------------
MetaMessage('track_name', name='test_rivers_flow', time=0)
MetaMessage('set_tempo', tempo=1000000, time=0)
MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0)
MetaMessage('track_name', name='Piano', time=0)
MetaMessage('track_name', name='Yamaha MOX', time=0)
note_on channel=0 note=71 velocity=66 time=2.85625
note_on channel=0 note=71 velocity=66 time=0.0020833333333333333
note_on channel=0 note=76 velocity=53 time=0.30416666666666664
note_o

In [583]:
midi_file_path = 'midi_data/hangyul_target2.midi'

# 미디 파일의 트랙 정보 출력
print_midi_info(midi_file_path)

result_df = midi_to_dataframe(midi_file_path)
result_df.to_csv('target2.csv', index=False)

print(result_df)

파일이름 : midi_data/hangyul_target2.midi
총 재생시간 : 12.424999999999997
----------------------------------------------------------------------------------------------------
트랙 이름 : test_rivers_flow
총 트랙의 수: 3
test_rivers_flow
Yamaha MOX
Yamaha MOX
----------------------------------------------------------------------------------------------------
트랙 1의 총 노트 수: 0
트랙 2의 총 노트 수: 108
트랙 3의 총 노트 수: 0
----------------------------------------------------------------------------------------------------
MetaMessage('track_name', name='test_rivers_flow', time=0)
MetaMessage('set_tempo', tempo=1000000, time=0)
MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0)
MetaMessage('track_name', name='Yamaha MOX', time=0)
MetaMessage('track_name', name='Yamaha MOX', time=0)
note_on channel=0 note=72 velocity=79 time=2.7604166666666665
note_on channel=1 note=72 velocity=79 time=0.0020833333333333333
note_on channel=0 note=76 velocity=59 time=0.333

# 2. 알고리즘 설계

## 2_1. 사전 작업
- 입력 데이터 파일(input_file)과 정답 데이터 파일(target_file)을 비교해서 정답 데이터 파일의 행 길이로 맞추기
- 만약에 입력 데이터 파일의 행 길이가 작으면 나머지는 0으로 처리

## 2_2. 노트 정확도(음정 정확도: pitch accuracy)
- 입력 데이터와 정답 데이터의 Notes를 비교하여 현재 노트가 일치하면 2점을 부여
- 구체화 필요
    - ex: 실수로 잘못 쳐서 한번 밀려서 연주한 경우 계속 밀리는 문제(-1, +1번째에 대해서도 검사를 할 것인지?) 

# 3. 알고리즘 구현

In [584]:
# 입력 데이터 및 정답 데이터 불러오기
input_file = 'hello1.csv'
input_file2 = 'hello3.csv'
input_file3 = 'hello4.csv'
target_file = 'hello2.csv'

input_hangyul = 'input.csv'
target_hangyul = 'target.csv'
input_hangyul1 = 'input2_1.csv'
input_hangyul2 = 'input2_2.csv'
input_hangyul3 = 'input2_3.csv'
target_hangyul = 'target.csv'
target_hangyul2 = 'target2.csv'

## 3_1. 두 파일의 길이 맞추기

In [585]:
def align_and_save_dataframes(input_file, target_file):
    df_input = pd.read_csv(input_file)
    df_target = pd.read_csv(target_file)

    # target_file의 행 길이로 맞추기
    min_length = min(len(df_input), len(df_target))
    df_input_aligned = df_input.head(min_length)

    # target_file보다 행이 작으면 나머지 행은 0으로 채우기
    if len(df_input) < len(df_target):
        df_input_aligned = pd.concat([df_input_aligned, pd.DataFrame(0, index=range(len(df_target) - len(df_input)), columns=df_input.columns)])

    # 모든 컬럼에 대해 0으로 채우기
    for col in df_input.columns:
        if col not in df_target.columns:
            df_input_aligned[col] = 0

    # msg_type, Notes, Velocities 컬럼에 대해 빈 리스트 []를 NaN으로 처리
    for col in ['msg_type', 'Notes', 'Velocities']:
        df_input_aligned[col] = df_input_aligned[col].apply(lambda x: np.nan if x == '[]' else x)

    df_input_aligned.to_csv(input_file, index=False)
    df_target.to_csv(target_file, index=False)

In [586]:
align_and_save_dataframes(input_file, target_file)
align_and_save_dataframes(input_file2, target_file)
align_and_save_dataframes(input_file3, target_file)
align_and_save_dataframes(input_hangyul, target_hangyul)
align_and_save_dataframes(input_hangyul1, target_hangyul2)
align_and_save_dataframes(input_hangyul2, target_hangyul2)
align_and_save_dataframes(input_hangyul3, target_hangyul2)

## 3_2. 노트 정확도(음정 정확도:pitch accuracy)

In [587]:
def calculate_note_accuracy(input_file, target_file):
    df_input = pd.read_csv(input_file)
    df_target = pd.read_csv(target_file)

    total_accuracy = 0

    for index in range(len(df_input)):
        notes_input = eval(df_input.at[index, 'Notes'])
        notes_target = eval(df_target.at[index, 'Notes'])

        print('-'*20)
        print(f'입력 데이터: {notes_input}')
        print(f'정답 데이터: {notes_target}')
        print('-'*20)
        
        # 0으로 된 값 리스트로 처리
        if isinstance(notes_input, int):
            notes_input = [notes_input]
        if isinstance(notes_target, int):
            notes_target = [notes_target]
        
        # 현재 행의 정확도 초기화
        row_accuracy = 0

        # 한 행에 있는 리스트들 간의 비교
        for i in range(len(notes_target)):
            for j in range(len(notes_input)):
                if notes_input[j] == notes_target[i]:
                    row_accuracy += 2
                    print('-'*10)
                    print(f'!!!{row_accuracy}점 추가!!!')
                    
                # 비교하는 대상들 출력
                print(f'비교 데이터 - 입력 데이터: {notes_input[j]} / 정답 데이터: {notes_target[i]}')

        # 한 행의 정확도를 전체 정확도에 추가
        total_accuracy += row_accuracy
        print(f'현재 점수: {total_accuracy}점')
        print()

    # 전체 정확도 계산
    max_possible_accuracy = 2 * len(df_input)
    accuracy_percentage = (total_accuracy / max_possible_accuracy) * 100

    return accuracy_percentage


In [588]:
accuracy = calculate_note_accuracy(input_file, target_file)
print(f'음정 정확도: {accuracy:.2f}%')

--------------------
입력 데이터: [60]
정답 데이터: [68]
--------------------
비교 데이터 - 입력 데이터: 60 / 정답 데이터: 68
현재 점수: 0점

--------------------
입력 데이터: [60, 62]
정답 데이터: [68, 70]
--------------------
비교 데이터 - 입력 데이터: 60 / 정답 데이터: 68
비교 데이터 - 입력 데이터: 62 / 정답 데이터: 68
비교 데이터 - 입력 데이터: 60 / 정답 데이터: 70
비교 데이터 - 입력 데이터: 62 / 정답 데이터: 70
현재 점수: 0점

--------------------
입력 데이터: [62, 64]
정답 데이터: [70, 68]
--------------------
비교 데이터 - 입력 데이터: 62 / 정답 데이터: 70
비교 데이터 - 입력 데이터: 64 / 정답 데이터: 70
비교 데이터 - 입력 데이터: 62 / 정답 데이터: 68
비교 데이터 - 입력 데이터: 64 / 정답 데이터: 68
현재 점수: 0점

--------------------
입력 데이터: [64, 65]
정답 데이터: [68, 66]
--------------------
비교 데이터 - 입력 데이터: 64 / 정답 데이터: 68
비교 데이터 - 입력 데이터: 65 / 정답 데이터: 68
비교 데이터 - 입력 데이터: 64 / 정답 데이터: 66
비교 데이터 - 입력 데이터: 65 / 정답 데이터: 66
현재 점수: 0점

--------------------
입력 데이터: [65, 67]
정답 데이터: [66, 68]
--------------------
비교 데이터 - 입력 데이터: 65 / 정답 데이터: 66
비교 데이터 - 입력 데이터: 67 / 정답 데이터: 66
비교 데이터 - 입력 데이터: 65 / 정답 데이터: 68
비교 데이터 - 입력 데이터: 67 / 정답 데이터: 68
현재 점수: 0점

------------

In [589]:
accuracy = calculate_note_accuracy(input_file2, target_file)
print(f'음정 정확도: {accuracy:.2f}%')

--------------------
입력 데이터: [60]
정답 데이터: [68]
--------------------
비교 데이터 - 입력 데이터: 60 / 정답 데이터: 68
현재 점수: 0점

--------------------
입력 데이터: [64]
정답 데이터: [68, 70]
--------------------
비교 데이터 - 입력 데이터: 64 / 정답 데이터: 68
비교 데이터 - 입력 데이터: 64 / 정답 데이터: 70
현재 점수: 0점

--------------------
입력 데이터: [67]
정답 데이터: [70, 68]
--------------------
비교 데이터 - 입력 데이터: 67 / 정답 데이터: 70
비교 데이터 - 입력 데이터: 67 / 정답 데이터: 68
현재 점수: 0점

--------------------
입력 데이터: [67, 72]
정답 데이터: [68, 66]
--------------------
비교 데이터 - 입력 데이터: 67 / 정답 데이터: 68
비교 데이터 - 입력 데이터: 72 / 정답 데이터: 68
비교 데이터 - 입력 데이터: 67 / 정답 데이터: 66
비교 데이터 - 입력 데이터: 72 / 정답 데이터: 66
현재 점수: 0점

--------------------
입력 데이터: [72, 76]
정답 데이터: [66, 68]
--------------------
비교 데이터 - 입력 데이터: 72 / 정답 데이터: 66
비교 데이터 - 입력 데이터: 76 / 정답 데이터: 66
비교 데이터 - 입력 데이터: 72 / 정답 데이터: 68
비교 데이터 - 입력 데이터: 76 / 정답 데이터: 68
현재 점수: 0점

--------------------
입력 데이터: [76, 67]
정답 데이터: [68, 77]
--------------------
비교 데이터 - 입력 데이터: 76 / 정답 데이터: 68
비교 데이터 - 입력 데이터: 67 / 정답 데이터: 68
비교 데이터 - 입

In [590]:
accuracy = calculate_note_accuracy(input_file3, target_file)
print(f'음정 정확도: {accuracy:.2f}%')

--------------------
입력 데이터: [41, 53, 57, 60, 64]
정답 데이터: [68]
--------------------
비교 데이터 - 입력 데이터: 41 / 정답 데이터: 68
비교 데이터 - 입력 데이터: 53 / 정답 데이터: 68
비교 데이터 - 입력 데이터: 57 / 정답 데이터: 68
비교 데이터 - 입력 데이터: 60 / 정답 데이터: 68
비교 데이터 - 입력 데이터: 64 / 정답 데이터: 68
현재 점수: 0점

--------------------
입력 데이터: [53]
정답 데이터: [68, 70]
--------------------
비교 데이터 - 입력 데이터: 53 / 정답 데이터: 68
비교 데이터 - 입력 데이터: 53 / 정답 데이터: 70
현재 점수: 0점

--------------------
입력 데이터: [57]
정답 데이터: [70, 68]
--------------------
비교 데이터 - 입력 데이터: 57 / 정답 데이터: 70
비교 데이터 - 입력 데이터: 57 / 정답 데이터: 68
현재 점수: 0점

--------------------
입력 데이터: [41]
정답 데이터: [68, 66]
--------------------
비교 데이터 - 입력 데이터: 41 / 정답 데이터: 68
비교 데이터 - 입력 데이터: 41 / 정답 데이터: 66
현재 점수: 0점

--------------------
입력 데이터: [60]
정답 데이터: [66, 68]
--------------------
비교 데이터 - 입력 데이터: 60 / 정답 데이터: 66
비교 데이터 - 입력 데이터: 60 / 정답 데이터: 68
현재 점수: 0점

--------------------
입력 데이터: [64]
정답 데이터: [68, 77]
--------------------
비교 데이터 - 입력 데이터: 64 / 정답 데이터: 68
비교 데이터 - 입력 데이터: 64 / 정답 데이터: 77
현재 점수:

In [591]:
accuracy = calculate_note_accuracy(input_file3, input_file2)
print()
print(f'음정 정확도: {accuracy:.2f}%')

--------------------
입력 데이터: [41, 53, 57, 60, 64]
정답 데이터: [60]
--------------------
비교 데이터 - 입력 데이터: 41 / 정답 데이터: 60
비교 데이터 - 입력 데이터: 53 / 정답 데이터: 60
비교 데이터 - 입력 데이터: 57 / 정답 데이터: 60
----------
!!!2점 추가!!!
비교 데이터 - 입력 데이터: 60 / 정답 데이터: 60
비교 데이터 - 입력 데이터: 64 / 정답 데이터: 60
현재 점수: 2점

--------------------
입력 데이터: [53]
정답 데이터: [64]
--------------------
비교 데이터 - 입력 데이터: 53 / 정답 데이터: 64
현재 점수: 2점

--------------------
입력 데이터: [57]
정답 데이터: [67]
--------------------
비교 데이터 - 입력 데이터: 57 / 정답 데이터: 67
현재 점수: 2점

--------------------
입력 데이터: [41]
정답 데이터: [67, 72]
--------------------
비교 데이터 - 입력 데이터: 41 / 정답 데이터: 67
비교 데이터 - 입력 데이터: 41 / 정답 데이터: 72
현재 점수: 2점

--------------------
입력 데이터: [60]
정답 데이터: [72, 76]
--------------------
비교 데이터 - 입력 데이터: 60 / 정답 데이터: 72
비교 데이터 - 입력 데이터: 60 / 정답 데이터: 76
현재 점수: 2점

--------------------
입력 데이터: [64]
정답 데이터: [76, 67]
--------------------
비교 데이터 - 입력 데이터: 64 / 정답 데이터: 76
비교 데이터 - 입력 데이터: 64 / 정답 데이터: 67
현재 점수: 2점

--------------------
입력 데이터: [41, 53, 56, 59, 

In [592]:
accuracy = calculate_note_accuracy(input_hangyul, target_hangyul)
print()
print(f'음정 정확도: {accuracy:.2f}%')

--------------------
입력 데이터: [80]
정답 데이터: [81, 54]
--------------------
비교 데이터 - 입력 데이터: 80 / 정답 데이터: 81
비교 데이터 - 입력 데이터: 80 / 정답 데이터: 54
현재 점수: 0점

--------------------
입력 데이터: [80]
정답 데이터: [81, 54, 80, 61]
--------------------
비교 데이터 - 입력 데이터: 80 / 정답 데이터: 81
비교 데이터 - 입력 데이터: 80 / 정답 데이터: 54
----------
!!!2점 추가!!!
비교 데이터 - 입력 데이터: 80 / 정답 데이터: 80
비교 데이터 - 입력 데이터: 80 / 정답 데이터: 61
현재 점수: 2점

--------------------
입력 데이터: [61]
정답 데이터: [80, 61, 81, 66]
--------------------
비교 데이터 - 입력 데이터: 61 / 정답 데이터: 80
----------
!!!2점 추가!!!
비교 데이터 - 입력 데이터: 61 / 정답 데이터: 61
비교 데이터 - 입력 데이터: 61 / 정답 데이터: 81
비교 데이터 - 입력 데이터: 61 / 정답 데이터: 66
현재 점수: 4점

--------------------
입력 데이터: [61]
정답 데이터: [81, 80]
--------------------
비교 데이터 - 입력 데이터: 61 / 정답 데이터: 81
비교 데이터 - 입력 데이터: 61 / 정답 데이터: 80
현재 점수: 4점

--------------------
입력 데이터: [61, 61]
정답 데이터: [80, 66, 81, 50]
--------------------
비교 데이터 - 입력 데이터: 61 / 정답 데이터: 80
비교 데이터 - 입력 데이터: 61 / 정답 데이터: 80
비교 데이터 - 입력 데이터: 61 / 정답 데이터: 66
비교 데이터 - 입력 데이터: 61 / 정답 데이

In [ ]:
accuracy = calculate_note_accuracy(input_hangyul1, target_hangyul2)
print()
print(f'음정 정확도: {accuracy:.2f}%')

In [ ]:
accuracy = calculate_note_accuracy(input_hangyul1, target_hangyul2)
print()
print(f'음정 정확도: {accuracy:.2f}%')